En repartant du dataset "people.csv" (cf. lesson4) dans sa version "clean" finale:

 1. mettre la colonne `inactive` à `true` pour tous les users dont le `last_seen` date d'au moins un an
 2. avec une regex: filtrer les numéros de téléphone invalides
 3. ajouter une colonne indiquant si le numéro de tel correspond à un téléphone portable (06/07)
 4. ajouter une colonne indiquant si les coordonnées GPS de l'utilisateur correspondent bien au "country"

In [1]:
import datetime
import pandas as pd
import re
import requests

In [2]:
# if you don't want pandas.read_csv to mess with data types,
# you can force it to keep str values by specifying dtype=str.
df = pd.read_csv('people.csv')

In [3]:
def clean_people(df):
    # rename columns:
    df = df.rename(columns={'email address': 'email'})
    
    # remove rows which have an empty "first_name" (NA):
    #df = df[df.first_name.notna()] <- equivalent to next line:
    df = df.dropna(subset=['first_name'])
    
    # drop duplicates on ID column:
    df = df.drop_duplicates()
    
    # Normalize gender column:
    df['gender'] = df['gender'].replace({'Female': 'F', 'Male': 'M'})
    
    # Convert column "age" to number (coerce: put NaN for bad values):
    df['age'] = pd.to_numeric(df.age, errors='coerce')
    
    # Convert columns to date type:
    df['registration'] = pd.to_datetime(df.registration)
    df['last_seen'] = pd.to_datetime(df.last_seen, unit='s')
    # When missing, last seen should fallback to the registration date:
    df['last_seen'] = df.last_seen.combine_first(df.registration)
    
    # Add a "full_name" column by concatenating two other ones:
    df['full_name'] = df.first_name + " " + df.last_name
    
    # Add a "country" column by extracting it from the address, with a split:
    df['country'] = df.address.str.split(', ').str[1]
    
    # Column "money" contains values like "$50.23" or "€23,09".
    # We want to make it uniform (only dollar currency) and as number, not str.
    df['currency'] = df.money.str[0]  # extract first char ($/€) to a new "currency" column
    df['money'] = df.money.str[1:].str.replace(',', '.')  # extract remaining chars and replace , by .
    df['money'] = pd.to_numeric(df.money)  # convert to number
    # convert euros cells to dollar:
    df.loc[df.currency == '€', 'money'] = df[df.currency == '€'].money * 1.10
    del df['currency']  # remove "currency" column which is now useless
    
    # Keep only rows where email is not NA:
    df = df.dropna(subset=['email'])
    # Keep only rows where email is a good email:
    # CAUTION: in the real world you should not use dummy regexes like this to validate email addresses,
    # but instead use a dedicated tool like https://github.com/syrusakbary/validate_email.
    df = df[df.email.str.contains('.+@[0-9a-zA-Z\.\-_]+\.\w{2,}')]
    # Some users may use email alias (example: john.smith+truc@gmail.com is an alias for john.smith@gmail.com).
    # We want to drop these duplicates. To do that, we extract the 'alias' part with a regex:
    groups = df.email.str.extract('([0-9a-zA-Z\.\-_]+)(\+[0-9a-zA-Z\.\-_]+)?(@[0-9a-zA-Z\.\-_]+\.\w{2,})')
    df['email'] = groups[0] + groups[2]  # we override the email with the email without the alias part
    # Then, just use drop_duplicates, which will keep the first line by default:
    df = df.drop_duplicates(subset=['email'])
    
    return df

df = clean_people(df)

In [4]:
df.head()

,id,first_name,last_name,email,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference,full_name,country
0,27625,Leandra,Pabelik,lpabelik5f@yale.edu,F,63.0,55.180,18.284100,49.632552,0136319724,2019-04-16,False,2019-06-03 12:39:57,"Palkovice, Czech Republic",entrée/plat/dessert,Leandra Pabelik,Czech Republic
1,27570,Ruthi,Ross,rross3w@sohu.com,F,57.0,20.370,19.230220,50.466575,NaN,2018-10-23,False,2019-08-30 11:41:04,"Siewierz, Poland",entrée/plat/dessert,Ruthi Ross,Poland
2,27572,Silas,Stourton,silas.stourton3y@answers.com,M,22.0,36.289,118.831081,24.984813,NaN,2018-12-30,False,2019-03-27 13:10:32,"Longbo, China",entrée/plat/dessert,Silas Stourton,China
3,27435,Roxine,Pettecrew,rpettecrew5@gravatar.com,F,64.0,108.823,121.648987,-8.844744,NaN,2019-03-12,False,2019-03-12 00:00:00,"Potulando, Indonesia",entrée/plat,Roxine Pettecrew,Indonesia
5,27497,Gordie,Bodicum,gbodicum1v@apache.org,M,60.0,47.260,13.978681,49.786243,0146943857,2017-04-29,False,2018-12-26 07:40:36,"Jince, Czech Republic",entrée/plat/dessert,Gordie Bodicum,Czech Republic


### 1. Mettre la colonne `inactive` à true pour tous les users dont le last_seen date d'au moins un an

In [5]:
# set inactive people
date_threshold = datetime.datetime.now() - datetime.timedelta(days=365)
df['inactive'] = df.last_seen < date_threshold

df.head()

,id,first_name,last_name,email,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference,full_name,country
0,27625,Leandra,Pabelik,lpabelik5f@yale.edu,F,63.0,55.180,18.284100,49.632552,0136319724,2019-04-16,False,2019-06-03 12:39:57,"Palkovice, Czech Republic",entrée/plat/dessert,Leandra Pabelik,Czech Republic
1,27570,Ruthi,Ross,rross3w@sohu.com,F,57.0,20.370,19.230220,50.466575,NaN,2018-10-23,False,2019-08-30 11:41:04,"Siewierz, Poland",entrée/plat/dessert,Ruthi Ross,Poland
2,27572,Silas,Stourton,silas.stourton3y@answers.com,M,22.0,36.289,118.831081,24.984813,NaN,2018-12-30,False,2019-03-27 13:10:32,"Longbo, China",entrée/plat/dessert,Silas Stourton,China
3,27435,Roxine,Pettecrew,rpettecrew5@gravatar.com,F,64.0,108.823,121.648987,-8.844744,NaN,2019-03-12,False,2019-03-12 00:00:00,"Potulando, Indonesia",entrée/plat,Roxine Pettecrew,Indonesia
5,27497,Gordie,Bodicum,gbodicum1v@apache.org,M,60.0,47.260,13.978681,49.786243,0146943857,2017-04-29,False,2018-12-26 07:40:36,"Jince, Czech Republic",entrée/plat/dessert,Gordie Bodicum,Czech Republic


### 2. avec une regex: filtrer les numéros de téléphone invalides

In [6]:
# filter valid phone number
pattern = '^0\d{9}$'

df = df[df.phone.str.contains(pattern, regex=True) == True]

df.head(20) 

,id,first_name,last_name,email,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference,full_name,country
0,27625,Leandra,Pabelik,lpabelik5f@yale.edu,F,63.0,55.180,18.284100,49.632552,0136319724,2019-04-16,False,2019-06-03 12:39:57,"Palkovice, Czech Republic",entrée/plat/dessert,Leandra Pabelik,Czech Republic
5,27497,Gordie,Bodicum,gbodicum1v@apache.org,M,60.0,47.260,13.978681,49.786243,0146943857,2017-04-29,False,2018-12-26 07:40:36,"Jince, Czech Republic",entrée/plat/dessert,Gordie Bodicum,Czech Republic
12,27548,Stanley,McCullen,smccullen3a@timesonline.co.uk,M,60.0,NaN,119.141473,28.074649,0875503094,2017-06-05,True,2017-06-05 00:00:00,"Longquan, China",entrée/plat/dessert,Stanley McCullen,China
15,27619,Shelley,Skeemor,sskeemor59@princeton.edu,F,44.0,41.500,111.878471,-7.584917,0547556034,2019-01-24,False,2019-05-04 13:18:24,"Guyangan, Indonesia",entrée/plat/dessert,Shelley Skeemor,Indonesia
16,27442,Mort,Midford,mmidfordc@yale.edu,M,59.0,NaN,6.848501,5.959253,0811321968,2019-06-25,False,2019-06-25 00:00:00,"Ozubulu, Nigeria",entrée/plat,Mort Midford,Nigeria
20,27462,Graig,Brownsea,gbrownseaw@arstechnica.com,M,54.0,80.230,110.393439,-7.213328,0262855559,2018-05-22,False,2019-09-28 13:28:10,"Sidomukti, Indonesia",entrée/plat/dessert,Graig Brownsea,Indonesia
24,27563,Andre,Lugsdin,alugsdin3p@microsoft.com,M,30.0,40.630,12.219712,57.121813,0144026028,2017-01-15,False,2019-06-03 02:11:38,"Varberg, Sweden",entrée/plat/dessert,Andre Lugsdin,Sweden
27,27551,Brigg,Bryett,bbryett3d@prlog.org,M,20.0,28.050,102.638296,24.301245,0746863267,2017-10-25,False,2019-04-10 01:46:26,"Dayingjie, China",entrée/plat/dessert,Brigg Bryett,China
30,27440,Erick,Meharry,emeharrya@wikispaces.com,M,52.0,44.792,3.575175,47.794456,0098166909,2017-04-21,True,2018-07-11 08:22:11,"Auxerre, France",entrée/plat/dessert,Erick Meharry,France
31,27593,Evelyn,Olenichev,eolenichev4j@tripadvisor.com,F,44.0,54.252,23.711756,37.944288,0214068650,2017-01-02,False,2019-08-07 22:59:39,"Néa Smýrni, Greece",entrée/plat/dessert,Evelyn Olenichev,Greece


### 3. Ajouter une colonne indiquant si le numéro de tel correspond à un téléphone portable (06/07)

In [7]:
df['is_mobile'] = df.phone.str.contains('^06|^07')

df.head()

,id,first_name,last_name,email,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference,full_name,country,is_mobile
0,27625,Leandra,Pabelik,lpabelik5f@yale.edu,F,63.0,55.18,18.284100,49.632552,0136319724,2019-04-16,False,2019-06-03 12:39:57,"Palkovice, Czech Republic",entrée/plat/dessert,Leandra Pabelik,Czech Republic,False
5,27497,Gordie,Bodicum,gbodicum1v@apache.org,M,60.0,47.26,13.978681,49.786243,0146943857,2017-04-29,False,2018-12-26 07:40:36,"Jince, Czech Republic",entrée/plat/dessert,Gordie Bodicum,Czech Republic,False
12,27548,Stanley,McCullen,smccullen3a@timesonline.co.uk,M,60.0,NaN,119.141473,28.074649,0875503094,2017-06-05,True,2017-06-05 00:00:00,"Longquan, China",entrée/plat/dessert,Stanley McCullen,China,False
15,27619,Shelley,Skeemor,sskeemor59@princeton.edu,F,44.0,41.50,111.878471,-7.584917,0547556034,2019-01-24,False,2019-05-04 13:18:24,"Guyangan, Indonesia",entrée/plat/dessert,Shelley Skeemor,Indonesia,False
16,27442,Mort,Midford,mmidfordc@yale.edu,M,59.0,NaN,6.848501,5.959253,0811321968,2019-06-25,False,2019-06-25 00:00:00,"Ozubulu, Nigeria",entrée/plat,Mort Midford,Nigeria,False


### 4. Ajouter une colonne indiquant si les coordonnées GPS de l'utilisateur correspondent bien au "country"

In [8]:
# country of IP address

def getCountryFromLonLat(x):

    result = None
    query = "https://geocode.xyz/{:.4f},{:.4f}?json=1".format(x[0], x[1])
    
    json = requests.get(query).json()

    if 'country' in json:
        result = json['country']

    return result


df["countryGuess"] = df[['lon','lat']].apply(getCountryFromLonLat, axis=1)

df.head()

https://geocode.xyz/18.2841,49.6326?json=1
https://geocode.xyz/13.9787,49.7862?json=1
https://geocode.xyz/119.1415,28.0746?json=1
https://geocode.xyz/111.8785,-7.5849?json=1
https://geocode.xyz/6.8485,5.9593?json=1
https://geocode.xyz/110.3934,-7.2133?json=1
https://geocode.xyz/12.2197,57.1218?json=1
https://geocode.xyz/102.6383,24.3012?json=1
https://geocode.xyz/3.5752,47.7945?json=1
https://geocode.xyz/23.7118,37.9443?json=1
https://geocode.xyz/112.8672,-6.9218?json=1
https://geocode.xyz/0.8133,46.6473?json=1
https://geocode.xyz/16.3042,57.9936?json=1
https://geocode.xyz/25.9111,49.1038?json=1
https://geocode.xyz/117.4301,23.7013?json=1
https://geocode.xyz/-39.0711,-11.2568?json=1
https://geocode.xyz/19.8833,40.9167?json=1
https://geocode.xyz/106.6173,-6.2956?json=1
https://geocode.xyz/20.2540,50.7481?json=1
https://geocode.xyz/15.5597,46.0733?json=1
https://geocode.xyz/110.6750,-7.6535?json=1
https://geocode.xyz/29.2916,-26.2155?json=1
https://geocode.xyz/3.2189,43.3363?json=1
https

,id,first_name,last_name,email,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference,full_name,country,is_mobile,countryGuess
0,27625,Leandra,Pabelik,lpabelik5f@yale.edu,F,63.0,55.18,18.284100,49.632552,0136319724,2019-04-16,False,2019-06-03 12:39:57,"Palkovice, Czech Republic",entrée/plat/dessert,Leandra Pabelik,Czech Republic,False,None
5,27497,Gordie,Bodicum,gbodicum1v@apache.org,M,60.0,47.26,13.978681,49.786243,0146943857,2017-04-29,False,2018-12-26 07:40:36,"Jince, Czech Republic",entrée/plat/dessert,Gordie Bodicum,Czech Republic,False,None
12,27548,Stanley,McCullen,smccullen3a@timesonline.co.uk,M,60.0,NaN,119.141473,28.074649,0875503094,2017-06-05,True,2017-06-05 00:00:00,"Longquan, China",entrée/plat/dessert,Stanley McCullen,China,False,None
15,27619,Shelley,Skeemor,sskeemor59@princeton.edu,F,44.0,41.50,111.878471,-7.584917,0547556034,2019-01-24,False,2019-05-04 13:18:24,"Guyangan, Indonesia",entrée/plat/dessert,Shelley Skeemor,Indonesia,False,None
16,27442,Mort,Midford,mmidfordc@yale.edu,M,59.0,NaN,6.848501,5.959253,0811321968,2019-06-25,False,2019-06-25 00:00:00,"Ozubulu, Nigeria",entrée/plat,Mort Midford,Nigeria,False,None


In [10]:
df.head(30)

,id,first_name,last_name,email,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference,full_name,country,is_mobile,countryGuess
0,27625,Leandra,Pabelik,lpabelik5f@yale.edu,F,63.0,55.180,18.284100,49.632552,0136319724,2019-04-16,False,2019-06-03 12:39:57,"Palkovice, Czech Republic",entrée/plat/dessert,Leandra Pabelik,Czech Republic,False,None
5,27497,Gordie,Bodicum,gbodicum1v@apache.org,M,60.0,47.260,13.978681,49.786243,0146943857,2017-04-29,False,2018-12-26 07:40:36,"Jince, Czech Republic",entrée/plat/dessert,Gordie Bodicum,Czech Republic,False,None
12,27548,Stanley,McCullen,smccullen3a@timesonline.co.uk,M,60.0,NaN,119.141473,28.074649,0875503094,2017-06-05,True,2017-06-05 00:00:00,"Longquan, China",entrée/plat/dessert,Stanley McCullen,China,False,None
15,27619,Shelley,Skeemor,sskeemor59@princeton.edu,F,44.0,41.500,111.878471,-7.584917,0547556034,2019-01-24,False,2019-05-04 13:18:24,"Guyangan, Indonesia",entrée/plat/dessert,Shelley Skeemor,Indonesia,False,None
16,27442,Mort,Midford,mmidfordc@yale.edu,M,59.0,NaN,6.848501,5.959253,0811321968,2019-06-25,False,2019-06-25 00:00:00,"Ozubulu, Nigeria",entrée/plat,Mort Midford,Nigeria,False,None
20,27462,Graig,Brownsea,gbrownseaw@arstechnica.com,M,54.0,80.230,110.393439,-7.213328,0262855559,2018-05-22,False,2019-09-28 13:28:10,"Sidomukti, Indonesia",entrée/plat/dessert,Graig Brownsea,Indonesia,False,None
24,27563,Andre,Lugsdin,alugsdin3p@microsoft.com,M,30.0,40.630,12.219712,57.121813,0144026028,2017-01-15,False,2019-06-03 02:11:38,"Varberg, Sweden",entrée/plat/dessert,Andre Lugsdin,Sweden,False,None
27,27551,Brigg,Bryett,bbryett3d@prlog.org,M,20.0,28.050,102.638296,24.301245,0746863267,2017-10-25,False,2019-04-10 01:46:26,"Dayingjie, China",entrée/plat/dessert,Brigg Bryett,China,True,None
30,27440,Erick,Meharry,emeharrya@wikispaces.com,M,52.0,44.792,3.575175,47.794456,0098166909,2017-04-21,True,2018-07-11 08:22:11,"Auxerre, France",entrée/plat/dessert,Erick Meharry,France,False,None
31,27593,Evelyn,Olenichev,eolenichev4j@tripadvisor.com,F,44.0,54.252,23.711756,37.944288,0214068650,2017-01-02,False,2019-08-07 22:59:39,"Néa Smýrni, Greece",entrée/plat/dessert,Evelyn Olenichev,Greece,False,None
